# Market Making Strategy

We simulate a market maker quoting around a midprice with constant spread.

Inventory control is used to manage risk exposure.

Profit & loss is computed with simulated fills.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

T = 300
mid_price = np.cumsum(np.random.randn(T)) + 100
spread = 0.5
bid = mid_price - spread / 2
ask = mid_price + spread / 2

inventory = np.zeros(T)
pnl = np.zeros(T)

for t in range(1, T):
    fill = np.random.choice([-1, 1])  # buy or sell
    inventory[t] = inventory[t-1] + fill
    pnl[t] = pnl[t-1] + fill * (mid_price[t] if fill == -1 else -mid_price[t])

plt.subplot(2,1,1)
plt.plot(inventory)
plt.title('Inventory Over Time')
plt.subplot(2,1,2)
plt.plot(pnl)
plt.title('PnL Over Time')
plt.tight_layout()
plt.show()